##### Copyright 2023 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Promoção de tipo do TF-NumPy

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/tf_numpy_type_promotion"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Há 4 opções para a promoção de tipo no TensorFlow.

- Por padrão, o TensorFlow gera erros em vez de promover tipos para operações de tipo misto.
- A execução de `tf.numpy.experimental_enable_numpy_behavior()` faz o TensorFlow trocar para as [regras de promoção de tipo do NumPy](https://www.tensorflow.org/guide/tf_numpy#type_promotion).
- **Este documento** descreve duas novas opções que estarão disponíveis no TensorFlow 2.15 (ou atualmente em `tf-nightly`):

In [ ]:
!pip install -q tf_nightly

**Observação**: `experimental_enable_numpy_behavior` muda o comportamento de todo o TensorFlow.

## Configuração

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.experimental.numpy as tnp

print("Using TensorFlow version %s" % tf.__version__)

### Ativando a nova promoção de tipo

Para usar a [promoção de tipo semelhante ao JAX](https://jax.readthedocs.io/en/latest/type_promotion.html) no TF-Numpy, especifique `'all'` ou `'safe'` como o modo de conversão dtype ao ativar o comportamento do NumPy para o TensorFlow.

Esse novo sistema (com `dtype_conversion_mode="all"`) é associativo, comutativo e facilita controlar a largura do float gerado (não converte automaticamente para floats mais largos). Ele apresenta alguns riscos de overflows e perda de precisão, mas `dtype_conversion_mode="safe"` força você a lidar com esses casos de maneira explícita. Os dois modos são explicados em mais detalhes na [próxima seção](#two_modes).

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")

<a name="two_modes">
</a>

## Dois modos: modo ALL x modo SAFE

No novo sistema de promoção de tipo, apresentamos dois modos: `ALL` e `SAFE`. O modo `SAFE` é usado para mitigar as preocupações de promoções "arriscadas" que podem resultar na perda de precisão ou no widening de bits.

### Dtypes

Vamos usar as seguintes abreviações por questões de brevidade.

- `b` significa `tf.bool`
- `u8` significa `tf.uint8`
- `i16` significa `tf.int16`
- `i32` significa `tf.int32`
- `bf16` significa `tf.bfloat16`
- `f32` significa `tf.float32`
- `f64` significa `tf.float64`
- `i32*` significa `int` do Python ou `i32` fracamente tipado
- `f32*` significa `float` do Python ou `f32` fracamente tipado
- `c128*` significa `complex` do Python ou `c128` fracamente tipado

O asterisco (*) indica que o tipo correspondente é "fraco": esse dtype é temporariamente inferido pelo sistema e pode ceder a outros dtypes. Esse conceito é explicado em mais detalhes [aqui](#weak_tensor).

### Exemplo de operações que perdem precisão

No exemplo a seguir, `i32` + `f32` é permitido no modo `ALL`, mas não no modo `SAFE`, devido ao risco de perda da precisão.

In [ ]:
# i32 + f32 returns a f32 result in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, dtype = tf.int32)
b = tf.constant(5.0, dtype = tf.float32)
a + b  # <tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [ ]:
# This promotion is not allowed in SAFE mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="safe")
a = tf.constant(10, dtype = tf.int32)
b = tf.constant(5.0, dtype = tf.float32)
try:
  a + b
except TypeError as e:
   print(f'{type(e)}: {e}')  # TypeError: explicitly specify the dtype or switch to ALL mode.

### Exemplo de operações com widening de bits

No exemplo a seguir, `i8` + `u32` é permitido no modo `ALL`, mas não no modo `SAFE`, devido ao widening de bits, ou seja, o uso de mais bits que o número de bits nas entradas. Observe que a semântica da nova promoção de tipo só permite o widening de bits necessário.

In [ ]:
# i8 + u32 returns an i64 result in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, dtype = tf.int8)
b = tf.constant(5, dtype = tf.uint32)
a + b

In [ ]:
# This promotion is not allowed in SAFE mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="safe")
a = tf.constant(10, dtype = tf.int8)
b = tf.constant(5, dtype = tf.uint32)
try:
  a + b
except TypeError as e:
   print(f'{type(e)}: {e}')  # TypeError: explicitly specify the dtype or switch to ALL mode.

## Um sistema baseado em lattice

### Lattice de promoção de tipo

O novo comportamento da promoção de tipo é determinado pelo seguinte lattice de promoção de tipo:

![Lattice de promoção de tipo](https://tensorflow.org/guide/images/new_type_promotion/type_promotion_lattice.png)

Mais especificamente, a promoção entre qualquer dois tipos é determinada ao encontrar o primeiro filho em comum de dois nós (incluindo os próprios nós).

Por exemplo, no diagrama acima, o primeiro filho em comum de `i8` e `i32` é `i32`, porque os dois nós se encontram pela primeira vez em `i32` ao seguir a direção das setas.

De maneira semelhante a outro exemplo, o tipo de promoção de resultado entre `u64` e `f16` seria `f16`.

<a name="promotion_table">
</a>

### Tabela de promoção de tipo

Ao seguir o lattice, é gerada a tabela de promoção binária abaixo:

**Observação**: o modo `SAFE` proíbe as células destacadas. O modo `ALL` permite todos os casos.

![Tabela de promoção de tipo](https://tensorflow.org/guide/images/new_type_promotion/type_promotion_table.png)

## Vantagens da nova promoção de tipo

Adotamos um sistema baseado em lattice semelhante ao JAX para nossa promoção de tipo, que oferece as seguintes vantagens:

<a name="lattice_system_design">
</a>

#### Vantagens do sistema baseado em lattice

Primeiro, o uso de um sistema baseado em lattice permite três propriedades muito importantes:

- Existência: há um tipo de promoção de resultado único para qualquer combinação de tipos.
- Comutatividade: `a + b = b + a`
- Associatividade: `a + (b + c) = (a + b) = c`

Essas três propriedades são fundamentais para construir uma semântica de promoção de tipo consistente e previsível.

#### Vantagens do sistema de lattice semelhante ao JAX

Outra vantagem crucial do sistema de lattice semelhante ao JAX é que, fora ints não assinados, ele evita todas as promoções maiores do que o necessário. Isso significa que você não pode obter resultados de 64 bits sem entradas de 64 bits. Isso é especialmente vantajoso para trabalhar com aceleradores, já que evita valores de 64 bits desnecessários, o que era frequente na promoção de tipo antiga.

No entanto, isso tem um lado ruim: a promoção de float/número inteiro mista é bastante propensa à perda de precisão. No exemplo abaixo, `i64` + `f16` resulta na promoção de `i64` a `f16`.

In [ ]:
# The first input is promoted to f16 in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
tf.constant(1, tf.int64) + tf.constant(3.2, tf.float16)  # <tf.Tensor: shape=(), dtype=float16, numpy=4.2>

Para reduzir essa preocupação, apresentamos um modo `SAFE` que proibirá essas promoções "arriscadas".

**Observação**: para saber mais sobre as considerações de design ao construir o sistema de lattice, consulte [Design da semântica de promoção de tipo para JAX](https://jax.readthedocs.io/en/latest/jep/9407-type-promotion.html).

<a name="weak_tensor">
</a>

## WeakTensor

### Visão geral

*Tensores fracos* são tensores "fracamente tipados", semelhante a um [conceito no JAX](https://jax.readthedocs.io/en/latest/type_promotion.html#weakly-typed-values-in-jax).

O dtype do `WeakTensor` é temporariamente inferido pelo sistema e pode ceder a outros dtypes. Esse conceito é apresentado na nova promoção de tipo para evitar a promoção de tipo indesejada em operações binárias entre valores do TF e valores sem tipo explicitamente especificado pelo usuário, como literais escalares do Python.

No exemplo abaixo, `tf.constant(1.2)` é considerado "fraco" porque não tem um dtype específico. Por isso, `tf.constant(1.2)` cede para o tipo `tf.constant(3.1, tf.float16)`, resultando na saída `f16`.

In [ ]:
tf.constant(1.2) + tf.constant(3.1, tf.float16)  # <tf.Tensor: shape=(), dtype=float16, numpy=4.3>

### Construção do WeakTensor

WeakTensors são gerados quando você cria um tensor sem especificar um dtype. Você pode conferir se um Tensor é "fraco" ou não ao verificar o atributo weak no final da representação de string do Tensor.

**Primeiro caso**: quando `tf.constant` é chamada com uma entrada sem dtype especificado pelo usuário.

In [ ]:
tf.constant(5)  # <tf.Tensor: shape=(), dtype=int32, numpy=5, weak=True>

In [ ]:
tf.constant([5.0, 10.0, 3])  # <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 5., 10.,  3.], dtype=float32), weak=True>

In [ ]:
# A normal Tensor is created when dtype arg is specified.
tf.constant(5, tf.int32)  # <tf.Tensor: shape=(), dtype=int32, numpy=5>

**Segundo caso**: quando uma entrada sem dtype especificado pelo usuário é passada para uma [API compatível com WeakTensors](#weak_tensor_apis).

In [ ]:
tf.math.abs([100.0, 4.0])  # <tf.Tensor: shape=(2,), dtype=float32, numpy=array([100., 4.], dtype=float32), weak=True>

##Efeitos da ativação da nova promoção de tipo

Confira abaixo uma lista não exaustiva das mudanças resultantes da ativação da nova promoção de tipo.

- Resultados de promoção mais consistentes e previsíveis.
- Menor risco de widening de bits.
- Os métodos mágicos matemáticos do `tf.Tensor` usam a nova promoção de tipo.
- `tf.constant` pode retornar `WeakTensor`.
- `tf.constant` permite conversões implícitas quando a entrada de um Tensor com um dtype diferente do arg `dtype` é passada.
- As operações in-place `tf.Variable` (`assign`, `assign-add` e `assign-sub`) permitem conversões implícitas.
- `tnp.array(1)` e `tnp.array(1.0)` retornam um WeakTensor de 32 bits.
- `WeakTensor`s serão criados e usados para [ APIs unárias e binárias compatíveis com WeakTensor](#weak_tensor_apis).


### Resultados de promoção mais consistentes e previsíveis

O uso de um [sistema baseado em lattice](#lattice_system_design) permite que a nova promoção de tipo produza resultados consistentes e previsíveis.

#### Promoção de tipo antiga

Com a promoção de tipo antiga, a mudança da ordem das operações produz resultados inconsistentes.

In [ ]:
# Setup
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="legacy")
a = np.array(1, dtype=np.int8)
b = tf.constant(1)
c = np.array(1, dtype=np.float16)

In [ ]:
# (a + b) + c throws an InvalidArgumentError.
try:
  tf.add(tf.add(a, b), c)
except tf.errors.InvalidArgumentError as e:
  print(f'{type(e)}: {e}')  # InvalidArgumentError

In [ ]:
# (b + a) + c returns an i32 result.
tf.add(tf.add(b, a), c)  # <tf.Tensor: shape=(), dtype=int32, numpy=3>

#### Nova promoção de tipo

A nova promoção de tipo produz resultados consistentes independentemente da ordem.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = np.array(1, dtype=np.int8)
b = tf.constant(1)
c = np.array(1, dtype=np.float16)

In [ ]:
# (a + b) + c returns a f16 result.
tf.add(tf.add(a, b), c)  # <tf.Tensor: shape=(), dtype=float16, numpy=3.0>

In [ ]:
# (b + a) + c also returns a f16 result.
tf.add(tf.add(b, a), c)  # <tf.Tensor: shape=(), dtype=float16, numpy=3.0>

### Menor risco de widening de bits

#### Promoção de tipo antiga

A promoção de tipo antiga frequentemente gerava resultados de 64 bits.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="legacy")

In [ ]:
np.array(3.2, np.float16) + tf.constant(1, tf.int8) + tf.constant(50)  # <tf.Tensor: shape=(), dtype=float64, numpy=54.19921875>

#### Nova promoção de tipo

A nova promoção de tipo retorna resultados com o número mínimo de bits necessário.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")

In [ ]:
np.array(3.2, np.float16) + tf.constant(1, tf.int8) + tf.constant(50)  # <tf.Tensor: shape=(), dtype=float16, numpy=54.2>

### Métodos mágicos matemáticos do tf.Tensor

Todos os métodos mágicos matemáticos do `tf.Tensor` seguirão a nova promoção de tipo.

In [ ]:
-tf.constant(5)  # <tf.Tensor: shape=(), dtype=int32, numpy=-5, weak=True>

In [ ]:
tf.constant(5, tf.int16) - tf.constant(1, tf.float32)  # <tf.Tensor: shape=(), dtype=float32, numpy=4.0>

### Ops in-place de tf.Variable

As conversões implícitas serão permitidas nas ops in-place de `tf.Variable`.

**Observação**: qualquer promoção que resultar em um dtype diferente do dtype original da variável não será permitida. Isso ocorre porque `tf.Variable` não pode mudar seu dtype.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.Variable(10, tf.int32)
a.assign_add(tf.constant(5, tf.int16))  # <tf.Variable shape=() dtype=int32, numpy=15>

### Conversões implícitas de tf.constant

Na antiga promoção de tipo, `tf.constant` exigia que um Tensor de entrada tivesse o mesmo dtype que o argumento dtype. No entanto, na nova promoção de tipo, convertemos implicitamente o Tensor para o dtype especificado.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, tf.int16)
tf.constant(a, tf.float32)  # <tf.Tensor: shape=(), dtype=float32, numpy=10.0>

### Array do TF-NumPy

`tnp.array` reverte a `i32*` e `f32*` para entradas do Python usando a nova promoção de tipo.

In [ ]:
tnp.array(1)  # <tf.Tensor: shape=(), dtype=int32, numpy=1, weak=True>

In [ ]:
tnp.array(1.0)  # <tf.Tensor: shape=(), dtype=int32, numpy=1, weak=True>

##Inferência de tipo da entrada

É assim que diferentes tipos de entradas são inferidos na nova promoção de tipo.

- `tf.Tensor`: como `tf.Tensor` tem uma propriedade dtype, não realizamos inferência adicional.
- Tipos do NumPy: isso inclui tipos como `np.array(1)`, `np.int16(1)` e `np.float`. Como as entradas do NumPy também tem uma propriedade dtype, tomamos a propriedade dtype como o tipo de inferência do resultado. Observe que o NumPy reverte a `i64` e `f64`.
- Tipos escalares/aninhados do Python: isso inclui tipos como `1`, `[1, 2, 3]` e `(1.0, 2.0)`.
    - `int` do Python é inferido como `i32*`.
    - `float` do Python é inferido como `f32*`.
    - `complex` do Python é inferido como `c128*`.
- Se a entrada não se enquadrar em nenhuma das categorias acima, mas tiver uma propriedade dtype, tomaremos a propriedade dtype como o tipo de inferência do resultado.

# Leituras adicionais

A nova promoção de tipo se assemelha muito à promoção de tipo do JAX-NumPy. Se você quiser saber mais detalhes sobre a nova promoção de tipo e as escolhas de design, confira os recursos abaixo.

- [Semântica de promoção de tipo do JAX](https://jax.readthedocs.io/en/latest/type_promotion.html)
- [Design da semântica de promoção de tipo para JAX](https://jax.readthedocs.io/en/latest/jep/9407-type-promotion.html)
- [Antiga semântica de promoção do TF-NumPy](https://www.tensorflow.org/guide/tf_numpy#type_promotion)


# Referências

<a name="weak_tensor_apis">
</a>

## APIs compatíveis com WeakTensor

Confira abaixo uma lista de APIs compatíveis com `WeakTensor`.

Para uma operação unária, isso significa que, se uma entrada sem tipo especificado pelo usuário for passada, ela retornará um `WeakTensor`.

Para uma op binária, ela seguirá esta [tabela de promoção](#promotion_table). Ela pode ou não retornar um `WeakTensor` dependendo do resultado da promoção das duas entradas.

**Observação**: todas as operações matemáticas (`+`, `-`, `*`, ...) são compatíveis.

- `tf.bitwise.invert`
- `tf.clip_by_value`
- `tf.debugging.check_numerics`
- `tf.expand_dims`
- `tf.identity`
- `tf.image.adjust_brightness`
- `tf.image.adjust_gamma`
- `tf.image.extract_patches`
- `tf.image.random_brightness`
- `tf.image.stateless_random_brightness`
- `tf.linalg.diag`
- `tf.linalg.diag_part`
- `tf.linalg.matmul`
- `tf.linalg.matrix_transpose`
- `tf.linalg.tensor_diag_part`
- `tf.linalg.trace`
- `tf.math.abs`
- `tf.math.acos`
- `tf.math.acosh`
- `tf.math.add`
- `tf.math.angle`
- `tf.math.asin`
- `tf.math.asinh`
- `tf.math.atan`
- `tf.math.atanh`
- `tf.math.ceil`
- `tf.math.conj`
- `tf.math.cos`
- `tf.math.cosh`
- `tf.math.digamma`
- `tf.math.divide_no_nan`
- `tf.math.divide`
- `tf.math.erf`
- `tf.math.erfc`
- `tf.math.erfcinv`
- `tf.math.erfinv`
- `tf.math.exp`
- `tf.math.expm1`
- `tf.math.floor`
- `tf.math.floordiv`
- `tf.math.floormod`
- `tf.math.imag`
- `tf.math.lgamma`
- `tf.math.log1p`
- `tf.math.log_sigmoid`
- `tf.math.log`
- `tf.math.multiply_no_nan`
- `tf.math.multiply`
- `tf.math.ndtri`
- `tf.math.negative`
- `tf.math.pow`
- `tf.math.real`
- `tf.math.real`
- `tf.math.reciprocal_no_nan`
- `tf.math.reciprocal`
- `tf.math.reduce_euclidean_norm`
- `tf.math.reduce_logsumexp`
- `tf.math.reduce_max`
- `tf.math.reduce_mean`
- `tf.math.reduce_min`
- `tf.math.reduce_prod`
- `tf.math.reduce_std`
- `tf.math.reduce_sum`
- `tf.math.reduce_variance`
- `tf.math.rint`
- `tf.math.round`
- `tf.math.rsqrt`
- `tf.math.scalar_mul`
- `tf.math.sigmoid`
- `tf.math.sign`
- `tf.math.sin`
- `tf.math.sinh`
- `tf.math.softplus`
- `tf.math.special.bessel_i0`
- `tf.math.special.bessel_i0e`
- `tf.math.special.bessel_i1`
- `tf.math.special.bessel_i1e`
- `tf.math.special.bessel_j0`
- `tf.math.special.bessel_j1`
- `tf.math.special.bessel_k0`
- `tf.math.special.bessel_k0e`
- `tf.math.special.bessel_k1`
- `tf.math.special.bessel_k1e`
- `tf.math.special.bessel_y0`
- `tf.math.special.bessel_y1`
- `tf.math.special.dawsn`
- `tf.math.special.expint`
- `tf.math.special.fresnel_cos`
- `tf.math.special.fresnel_sin`
- `tf.math.special.spence`
- `tf.math.sqrt`
- `tf.math.square`
- `tf.math.subtract`
- `tf.math.tan`
- `tf.math.tanh`
- `tf.nn.depth_to_space`
- `tf.nn.elu`
- `tf.nn.gelu`
- `tf.nn.leaky_relu`
- `tf.nn.log_softmax`
- `tf.nn.relu6`
- `tf.nn.relu`
- `tf.nn.selu`
- `tf.nn.softsign`
- `tf.nn.space_to_depth`
- `tf.nn.swish`
- `tf.ones_like`
- `tf.realdiv`
- `tf.reshape`
- `tf.squeeze`
- `tf.stop_gradient`
- `tf.transpose`
- `tf.truncatediv`
- `tf.truncatemod`
- `tf.zeros_like`
- `tf.experimental.numpy.abs`
- `tf.experimental.numpy.absolute`
- `tf.experimental.numpy.amax`
- `tf.experimental.numpy.amin`
- `tf.experimental.numpy.angle`
- `tf.experimental.numpy.arange`
- `tf.experimental.numpy.arccos`
- `tf.experimental.numpy.arccosh`
- `tf.experimental.numpy.arcsin`
- `tf.experimental.numpy.arcsinh`
- `tf.experimental.numpy.arctan`
- `tf.experimental.numpy.arctanh`
- `tf.experimental.numpy.around`
- `tf.experimental.numpy.array`
- `tf.experimental.numpy.asanyarray`
- `tf.experimental.numpy.asarray`
- `tf.experimental.numpy.ascontiguousarray`
- `tf.experimental.numpy.average`
- `tf.experimental.numpy.bitwise_not`
- `tf.experimental.numpy.cbrt`
- `tf.experimental.numpy.ceil`
- `tf.experimental.numpy.conj`
- `tf.experimental.numpy.conjugate`
- `tf.experimental.numpy.copy`
- `tf.experimental.numpy.cos`
- `tf.experimental.numpy.cosh`
- `tf.experimental.numpy.cumprod`
- `tf.experimental.numpy.cumsum`
- `tf.experimental.numpy.deg2rad`
- `tf.experimental.numpy.diag`
- `tf.experimental.numpy.diagflat`
- `tf.experimental.numpy.diagonal`
- `tf.experimental.numpy.diff`
- `tf.experimental.numpy.empty_like`
- `tf.experimental.numpy.exp2`
- `tf.experimental.numpy.exp`
- `tf.experimental.numpy.expand_dims`
- `tf.experimental.numpy.expm1`
- `tf.experimental.numpy.fabs`
- `tf.experimental.numpy.fix`
- `tf.experimental.numpy.flatten`
- `tf.experimental.numpy.flip`
- `tf.experimental.numpy.fliplr`
- `tf.experimental.numpy.flipud`
- `tf.experimental.numpy.floor`
- `tf.experimental.numpy.full_like`
- `tf.experimental.numpy.imag`
- `tf.experimental.numpy.log10`
- `tf.experimental.numpy.log1p`
- `tf.experimental.numpy.log2`
- `tf.experimental.numpy.log`
- `tf.experimental.numpy.max`
- `tf.experimental.numpy.mean`
- `tf.experimental.numpy.min`
- `tf.experimental.numpy.moveaxis`
- `tf.experimental.numpy.nanmean`
- `tf.experimental.numpy.negative`
- `tf.experimental.numpy.ones_like`
- `tf.experimental.numpy.positive`
- `tf.experimental.numpy.prod`
- `tf.experimental.numpy.rad2deg`
- `tf.experimental.numpy.ravel`
- `tf.experimental.numpy.real`
- `tf.experimental.numpy.reciprocal`
- `tf.experimental.numpy.repeat`
- `tf.experimental.numpy.reshape`
- `tf.experimental.numpy.rot90`
- `tf.experimental.numpy.round`
- `tf.experimental.numpy.signbit`
- `tf.experimental.numpy.sin`
- `tf.experimental.numpy.sinc`
- `tf.experimental.numpy.sinh`
- `tf.experimental.numpy.sort`
- `tf.experimental.numpy.sqrt`
- `tf.experimental.numpy.square`
- `tf.experimental.numpy.squeeze`
- `tf.experimental.numpy.std`
- `tf.experimental.numpy.sum`
- `tf.experimental.numpy.swapaxes`
- `tf.experimental.numpy.tan`
- `tf.experimental.numpy.tanh`
- `tf.experimental.numpy.trace`
- `tf.experimental.numpy.transpose`
- `tf.experimental.numpy.triu`
- `tf.experimental.numpy.vander`
- `tf.experimental.numpy.var`
- `tf.experimental.numpy.zeros_like`